In [1]:
DATA_PATH = '../../../Toys/Recommender/ml-20m/' # '../../../Toys/Recommender/Bake-off/' # 
MOVIES_CSV = DATA_PATH + 'movies.csv' # 'bakeoff_movies.csv' #
USERS_CSV = DATA_PATH + 'users.csv' #'bakeoff_train_users.csv' # 
RATINGS_CSV = DATA_PATH + 'ratings.csv' # 'bakeoff_train_ratings.csv' #

In [2]:
from autorecommender.data import Dataset, ratings_matrix_to_list
from autorecommender.model import Autorecommender, load_model

/home/matthew/anaconda2/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np
import math

In [4]:
# USERS = pd.read_csv(USERS_CSV).set_index('userId').sort_index()
rating_counts = pd.read_csv(DATA_PATH + 'rating_counts.csv')
RATINGS = pd.read_csv(RATINGS_CSV)

movie_bucket = rating_counts[rating_counts['num_of_ratings'] > rating_counts['num_of_ratings'].mean()]['movieId']
RATINGS = RATINGS[RATINGS['movieId'].isin(movie_bucket)]
RATINGS = RATINGS.set_index(['userId', 'movieId']).sort_index()[['rating']]

MOVIES = pd.read_csv(MOVIES_CSV)
MOVIES = MOVIES[MOVIES['movieId'].isin(movie_bucket)]
MOVIES = MOVIES.set_index('movieId').sort_index()

USERS = pd.DataFrame({'userId': pd.read_csv(RATINGS_CSV)['userId'].unique()}).set_index('userId')

In [5]:
dataset = Dataset(MOVIES, USERS, RATINGS, userid='userId', itemid='movieId', ratingid='rating')

In [6]:
dataset.preferences.min()

-1.0

In [7]:
len(dataset.items)

3696

In [8]:
train_dataset, test_dataset = dataset.train_test_split(frac=0.9)

del dataset

In [9]:
model = Autorecommender(item_count=len(train_dataset.items))

In [10]:
model.fit(train_dataset, batch_size=64, epochs=1, validation_data=test_dataset, patience=0)

/home/matthew/anaconda2/envs/python3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
1947/1947 [==============================] - 3802s 2s/step - loss: 0.0274 - val_loss: 0.0270


In [11]:
model.save('model.zip')

In [ ]:
model = load_model('model.zip')

In [12]:
val_ratings = test_dataset.ratings

In [13]:
val_ratings.head()

rating
userId movieId        
6      1           5.0
       3           3.0
       7           5.0
       17          5.0
       52          5.0

In [14]:
val_ratings_x = val_ratings.groupby(level=0).apply(lambda x: x.sample(frac=0.8)).reset_index(level=0, drop=True)

In [15]:
len(val_ratings_x)/len(val_ratings)

0.8000550776403866

In [16]:
val_ratings_x.head()

rating
userId movieId        
6      260         4.0
       648         5.0
       736         2.0
       733         3.0
       743         3.0

In [17]:
val_idx_X = val_ratings_x.index
val_idx_y = val_ratings.index
val_idx = val_idx_y.difference(val_idx_X)

val_masked_ratings = val_ratings.loc[val_idx]

In [18]:
val_predictions = model.predict(val_ratings_x)

val_model_ratings = ratings_matrix_to_list(val_predictions).loc[val_idx]
VAL_MSE = ((val_masked_ratings - val_model_ratings)**2).mean()['rating']
VAL_MAE = (abs(val_masked_ratings - val_model_ratings)).mean()['rating']
VAL_RMSE = math.sqrt(VAL_MSE)

print('Bake-off RMSE is: {}, MAE is: {}'.format(VAL_RMSE, VAL_MAE))

Bake-off RMSE is: 0.8543093221012416, MAE is: 0.654712543112198


In [19]:
val_predictions = model.predict(val_ratings_x, null=True)

val_model_ratings = ratings_matrix_to_list(val_predictions).loc[val_idx]
VAL_MSE = ((val_masked_ratings - val_model_ratings)**2).mean()['rating']
VAL_MAE = (abs(val_masked_ratings - val_model_ratings)).mean()['rating']
VAL_RMSE = math.sqrt(VAL_MSE)

print('Bake-off RMSE is: {}, MAE is: {}'.format(VAL_RMSE, VAL_MAE))

Bake-off RMSE is: 0.8647882993920402, MAE is: 0.6574969401870406


In [ ]:
# model = load_model('model.zip')

In [ ]:
VAL_RATINGS = pd.read_csv(DATA_PATH + 'SECRET_bakeoff_validation_ratings.csv').set_index(['userid', 'movieid']).sort_index()[['rating']]
VAL_RATINGS_X = pd.read_csv(DATA_PATH + 'bakeoff_validation_ratings.csv').set_index(['userid', 'movieid']).sort_index()[['rating']]

val_idx_X = VAL_RATINGS_X.index
val_idx_y = VAL_RATINGS.index
val_idx = val_idx_y.difference(val_idx_X)

val_masked_ratings = VAL_RATINGS.loc[val_idx]

In [ ]:
val_predictions = model.predict(VAL_RATINGS_X)

val_model_ratings = ratings_matrix_to_list(val_predictions).loc[val_idx]
VAL_MSE = ((val_masked_ratings - val_model_ratings)**2).mean()['rating']
VAL_MAE = (abs(val_masked_ratings - val_model_ratings)).mean()['rating']
VAL_RMSE = math.sqrt(VAL_MSE)

print('Bake-off RMSE is: {}, MAE is: {}'.format(VAL_RMSE, VAL_MAE))

In [ ]:
val_predictions.loc[[3032, 4486, 5420]][[1470]]

In [ ]:
val_null_predictions = model.predict(VAL_RATINGS_X, null=True)

val_null_ratings = ratings_matrix_to_list(val_null_predictions).loc[val_idx]
VAL_MSE = ((val_masked_ratings - val_null_ratings)**2).mean()['rating']
VAL_MAE = (abs(val_masked_ratings - val_null_ratings)).mean()['rating']
VAL_RMSE = math.sqrt(VAL_MSE)

print('Bake-off RMSE is: {}, MAE is: {}'.format(VAL_RMSE, VAL_MAE))

In [ ]:
MOVIES.head()

In [ ]:
MOVIES.head()['genres'].str.split('|', expand=True).stack().str.get_dummies().groupby(level=0).sum()

In [ ]:
rating_counts = RATINGS.groupby(level=1).count().sort_values('rating', ascending=False)

In [ ]:
rating_counts.head()

In [ ]:
MOVIES.loc[296]

In [ ]:
MOVIES.loc[356]

In [ ]:
rating_counts.mean()

In [ ]:
rating_counts.median()

In [ ]:
rating_counts[rating_counts > 747].count()

In [ ]:
rating_counts.columns = ['num_of_ratings']

In [ ]:
rating_counts.head()

In [ ]:
rating_counts.to_csv(DATA_PATH + 'rating_counts.csv')

In [ ]:
DATA_PATH

In [ ]:
rating_counts2 = pd.read_csv(DATA_PATH + 'rating_counts.csv')

In [ ]:
rating_counts2.head()